In [1]:
import pandas as pd
import numpy as np


In [2]:
train = pd.read_csv("train.csv",encoding='big5')
test = pd.read_csv("test.csv",encoding='big5')

c:\Users\Julie\Documents\Python\.venv\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (17,21,22) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
train.info()#sample有100000

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Columns: 132 entries, CUS_ID to Y1
dtypes: float64(30), int64(12), object(90)
memory usage: 100.7+ MB


In [4]:
test.info()#sample有150000

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Columns: 131 entries, CUS_ID to FINANCETOOLS_G
dtypes: float64(30), int64(12), object(89)
memory usage: 149.9+ MB


# 前處理

In [5]:
dataset=pd.concat([train,test])#將train&test組合
dataset = dataset.reset_index(drop=True)#重新排列順序

In [6]:
#先將Y1轉換成label
dataset['Y1']=dataset['Y1'].replace('N',0)
dataset['Y1']=dataset['Y1'].replace('Y',1)

In [7]:
#砍掉Y1
data=dataset.iloc[:,:-1]
data.head()

,CUS_ID,GENDER,AGE,CHARGE_CITY_CD,CONTACT_CITY_CD,EDUCATION_CD,MARRIAGE_CD,LAST_A_CCONTACT_DT,L1YR_A_ISSUE_CNT,LAST_A_ISSUE_DT,...,IF_ADD_INSD_IND,L1YR_GROSS_PRE_AMT,CUST_9_SEGMENTS_CD,FINANCETOOLS_A,FINANCETOOLS_B,FINANCETOOLS_C,FINANCETOOLS_D,FINANCETOOLS_E,FINANCETOOLS_F,FINANCETOOLS_G
0,3418,M,低,A1,A1,NaN,NaN,Y,0,N,...,N,0.000174,C,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4302,M,低,A1,A1,NaN,NaN,Y,0,N,...,N,0.008724,A,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,5545,M,低,A1,A1,1.0,0.0,Y,1,Y,...,N,0.005359,A,Y,N,N,N,N,N,N
3,7207,M,低,A1,A1,NaN,0.0,Y,0,N,...,N,0.000000,C,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,7213,M,低,A1,A1,1.0,0.0,N,0,N,...,N,0.000603,C,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
#拉出Y1
y=dataset.iloc[:,-1]
y.head()

0    0.0
1    0.0
2    0.0
3    0.0
4    0.0
Name: Y1, dtype: float64

In [9]:
#缺失值佔整個feature的比率
all_data_na = (data.isnull().sum() /len(data) ) * 100
all_data_na = all_data_na.drop(all_data_na[all_data_na == 0].index).sort_values(ascending=False)[:100]
missing_data = pd.DataFrame({'Missing Ratio' :all_data_na})
missing_data

,Missing Ratio
L1YR_C_CNT,88.0312
A_IND,79.3968
C_IND,79.3968
B_IND,79.3968
FINANCETOOLS_A,62.4648
...,...
X_A_IND,0.0208
X_F_IND,0.0208
X_H_IND,0.0208
X_G_IND,0.0208


In [10]:
#show出有缺失值的feature
data[data.isnull().any(axis=1)].head().T

,0,1,2,3,4
CUS_ID,3418,4302,5545,7207,7213
GENDER,M,M,M,M,M
AGE,低,低,低,低,低
CHARGE_CITY_CD,A1,A1,A1,A1,A1
CONTACT_CITY_CD,A1,A1,A1,A1,A1
...,...,...,...,...,...
FINANCETOOLS_C,NaN,NaN,N,NaN,NaN
FINANCETOOLS_D,NaN,NaN,N,NaN,NaN
FINANCETOOLS_E,NaN,NaN,N,NaN,NaN
FINANCETOOLS_F,NaN,NaN,N,NaN,NaN


## 處理missing value

In [11]:
#推測不是重要feature，先以眾數補之
data["GENDER"].fillna(data["GENDER"].mode()[0], inplace = True)
#推測1=高中，2=研究所，3=大學，4=國中(9年義務教育)，Na補眾數=3
data['EDUCATION_CD'].fillna(3, inplace = True)
#未找到特別關聯，先以眾數補之
data["MARRIAGE_CD"].fillna(data["MARRIAGE_CD"].mode()[0], inplace = True)

In [12]:
#未能推算如何神秘轉換，Na先補0
data["APC_1ST_YEARDIF"].fillna(0, inplace = True)

In [13]:
#未找到特別關聯，先以眾數補之
data["OCCUPATION_CLASS_CD"].fillna(data["OCCUPATION_CLASS_CD"].mode()[0], inplace = True)
data["APC_1ST_AGE"].fillna(data["APC_1ST_AGE"].mode()[0], inplace = True)
data["INSD_1ST_AGE"].fillna(data['INSD_1ST_AGE'].mode()[0], inplace = True)
#未找到特別關聯，先以眾數補之
#等級越高，Y1=Y的機率越低，對分類做label
data["RFM_R"].fillna(data['RFM_R'].mode()[0], inplace = True)
data['RFM_R'] = data['RFM_R'].replace('高', 0)
data['RFM_R'] = data['RFM_R'].replace('中高', 1)
data['RFM_R'] = data['RFM_R'].replace('中', 2)
data['RFM_R'] = data['RFM_R'].replace('低', 3)
#未找到特別關聯，先以眾數補之
#等級越高，Y1=Y的機率越高，對分類做label
data["REBUY_TIMES_CNT"].fillna(data['REBUY_TIMES_CNT'].mode()[0], inplace = True)
data['REBUY_TIMES_CNT'] = data['REBUY_TIMES_CNT'].replace('高', 3)
data['REBUY_TIMES_CNT'] = data['REBUY_TIMES_CNT'].replace('中高', 2)
data['REBUY_TIMES_CNT'] = data['REBUY_TIMES_CNT'].replace('中', 1)
data['REBUY_TIMES_CNT'] = data['REBUY_TIMES_CNT'].replace('低', 0)
#未找到特別關聯，先以眾數補之
data["LEVEL"].fillna(data['LEVEL'].mode()[0], inplace = True)
data["RFM_M_LEVEL"].fillna(data['RFM_M_LEVEL'].mode()[0], inplace = True)
data["TERMINATION_RATE"].fillna(0, inplace = True)

In [14]:
#未能推算如何神秘轉換，Na先補0
data["ANNUAL_PREMIUM_AMT"].fillna(0, inplace = True)
data["ANNUAL_INCOME_AMT"].fillna(0, inplace = True)

In [15]:
#Y1=Y時，$_IND=Y的比重太小，故選眾數=N
#Na只有52個，可考慮刪除
data["A_IND"].fillna('N', inplace = True)
data["B_IND"].fillna('N', inplace = True)
data["C_IND"].fillna('N', inplace = True)

In [16]:
#三年內皆無到C通路申辦(LAST_C_DT=N)，L1YR_C_CNT就不會有申辦次數，所以Na=0
#三年內若有到C通路申辦(LAST_C_DT=N)，L1YR_C_CNT的眾數為1，估計近一年至少有一次到訪，所以Na=1
data.loc[data['LAST_C_DT']=='N','L1YR_C_CNT'] = 0
data['L1YR_C_CNT'].fillna(1, inplace = True)

In [17]:
#未能推算如何神秘轉換，Na先補0(眾數)
data["INSD_LAST_YEARDIF_CNT"].fillna(0, inplace = True)
#未能推算如何神秘轉換，但數據有級距，先以眾數補之
data["BMI"].fillna(data["BMI"].mode()[0], inplace = True)

In [18]:
#未能推算如何神秘轉換，Na先補0
data["DIEBENEFIT_AMT"].fillna(0, inplace = True)
data["DIEACCIDENT_AMT"].fillna(0, inplace = True)
data["POLICY_VALUE_AMT"].fillna(0, inplace = True)
data["ANNUITY_AMT"].fillna(0, inplace = True)
data["EXPIRATION_AMT"].fillna(0, inplace = True)
data["ACCIDENT_HOSPITAL_REC_AMT"].fillna(0, inplace = True)
data["DISEASES_HOSPITAL_REC_AMT"].fillna(0, inplace = True)
data["OUTPATIENT_SURGERY_AMT"].fillna(0, inplace = True)
data["INPATIENT_SURGERY_AMT"].fillna(0, inplace = True)
data["PAY_LIMIT_MED_MISC_AMT"].fillna(0, inplace = True)
data["FIRST_CANCER_AMT"].fillna(0, inplace = True)
data["ILL_ACCELERATION_AMT"].fillna(0, inplace = True)
data["ILL_ADDITIONAL_AMT"].fillna(0, inplace = True)
data["LONG_TERM_CARE_AMT"].fillna(0, inplace = True)
data["MONTHLY_CARE_AMT"].fillna(0, inplace = True)

In [19]:
capital = ['A','B','C','D','E','F','G','H']
X_IND = []
for i in range(0,len(capital)):
    X_IND.append('X_' + str(capital[i]) + '_IND')
print(X_IND)
#X_$_IND=Y時，Y1=Y的機率極低，全X_$_IND都=Y時也是，估計關連不大，故選眾數N
for i in range(0,len(X_IND)):
    data[X_IND[i]].fillna('N', inplace = True)

['X_A_IND', 'X_B_IND', 'X_C_IND', 'X_D_IND', 'X_E_IND', 'X_F_IND', 'X_G_IND', 'X_H_IND']


In [20]:
capital = ['A','B','C','D','E','F','G','H','I','J','K','L','M','N','O','P','Q']
IF_ISSUE_INSD_IND = []
for i in range(0,len(capital)):
    IF_ISSUE_INSD_IND.append('IF_ISSUE_INSD_' + str(capital[i]) + '_IND')
print(IF_ISSUE_INSD_IND)
#未找到特別關聯，先以眾數補之
for i in range(0,len(IF_ISSUE_INSD_IND)):
    data[IF_ISSUE_INSD_IND[i]].fillna(data[IF_ISSUE_INSD_IND[i]].mode()[0], inplace = True)

['IF_ISSUE_INSD_A_IND', 'IF_ISSUE_INSD_B_IND', 'IF_ISSUE_INSD_C_IND', 'IF_ISSUE_INSD_D_IND', 'IF_ISSUE_INSD_E_IND', 'IF_ISSUE_INSD_F_IND', 'IF_ISSUE_INSD_G_IND', 'IF_ISSUE_INSD_H_IND', 'IF_ISSUE_INSD_I_IND', 'IF_ISSUE_INSD_J_IND', 'IF_ISSUE_INSD_K_IND', 'IF_ISSUE_INSD_L_IND', 'IF_ISSUE_INSD_M_IND', 'IF_ISSUE_INSD_N_IND', 'IF_ISSUE_INSD_O_IND', 'IF_ISSUE_INSD_P_IND', 'IF_ISSUE_INSD_Q_IND']


In [21]:
capital = ['F','L','Q','G','R']
IF_ADD_INSD_IND = []
for i in range(0,len(capital)):
    IF_ADD_INSD_IND.append('IF_ADD_INSD_' + str(capital[i]) + '_IND')
print(IF_ADD_INSD_IND)
#未找到特別關聯，先以眾數補之
for i in range(0,len(IF_ADD_INSD_IND)):
    data[IF_ADD_INSD_IND[i]].fillna(data[IF_ADD_INSD_IND[i]].mode()[0], inplace = True)

['IF_ADD_INSD_F_IND', 'IF_ADD_INSD_L_IND', 'IF_ADD_INSD_Q_IND', 'IF_ADD_INSD_G_IND', 'IF_ADD_INSD_R_IND']


In [22]:
#未找到特別關聯，先以眾數補之
#Na只有172個，可考慮刪除
data['IF_ADD_INSD_IND'].fillna(data['IF_ADD_INSD_IND'].mode()[0], inplace = True)

In [23]:
#Y1=N、Y時，FINANCETOOLS_A的Y都最多
data["FINANCETOOLS_A"].fillna('Y', inplace = True)
#Y1=N、Y時，FINANCETOOLS_$的N都最多
data["FINANCETOOLS_B"].fillna('N', inplace = True)
data["FINANCETOOLS_C"].fillna('N', inplace = True)
data["FINANCETOOLS_D"].fillna('N', inplace = True)
data["FINANCETOOLS_E"].fillna('N', inplace = True)
data["FINANCETOOLS_F"].fillna('N', inplace = True)
data["FINANCETOOLS_G"].fillna('N', inplace = True)

In [24]:
all_data_na = (data.isnull().sum() /len(data) ) * 100
all_data_na = all_data_na.drop(all_data_na[all_data_na == 0].index).sort_values(ascending=False)[:100]
missing_data = pd.DataFrame({'Missing Ratio' :all_data_na})
missing_data

,Missing Ratio


# 做labal&onehot轉換

In [25]:
#等級越高，Y1=Y的機率越高，對分類做label
data['LIFE_CNT'] = data['LIFE_CNT'].replace('高', 2)
data['LIFE_CNT'] = data['LIFE_CNT'].replace('中', 1)
data['LIFE_CNT'] = data['LIFE_CNT'].replace('低', 0)

In [26]:
onehot_encoder_feature = ['GENDER','AGE','CHARGE_CITY_CD','CONTACT_CITY_CD','EDUCATION_CD','MARRIAGE_CD',\
                          'OCCUPATION_CLASS_CD','APC_1ST_AGE','INSD_1ST_AGE','IM_CNT','CUST_9_SEGMENTS_CD']

In [27]:
#抓出data裡屬於object的column
object_column = data.loc[:,data.dtypes==object].head()
label_encoder_feature = list(object_column)

In [28]:
#LEVEL&RFM_M_LEVEL要做label，但為數值，為抓取到
label_encoder_feature.append('LEVEL')
label_encoder_feature.append('RFM_M_LEVEL')
len(label_encoder_feature)

88

In [29]:
#把label_encoder_feature裡要做onehot的feature刪除
column = ['GENDER','AGE','CHARGE_CITY_CD','CONTACT_CITY_CD','APC_1ST_AGE','INSD_1ST_AGE','CUST_9_SEGMENTS_CD']
for i  in range(0,len(column)):
    label_encoder_feature.remove(column[i])
label_encoder_feature

['LAST_A_CCONTACT_DT',
 'LAST_A_ISSUE_DT',
 'LAST_B_ISSUE_DT',
 'IF_2ND_GEN_IND',
 'IF_ISSUE_A_IND',
 'IF_ISSUE_B_IND',
 'IF_ISSUE_C_IND',
 'IF_ISSUE_D_IND',
 'IF_ISSUE_E_IND',
 'IF_ISSUE_F_IND',
 'IF_ISSUE_G_IND',
 'IF_ISSUE_H_IND',
 'IF_ISSUE_I_IND',
 'IF_ISSUE_J_IND',
 'IF_ISSUE_K_IND',
 'IF_ISSUE_L_IND',
 'IF_ISSUE_M_IND',
 'IF_ISSUE_N_IND',
 'IF_ISSUE_O_IND',
 'IF_ISSUE_P_IND',
 'IF_ISSUE_Q_IND',
 'IF_ADD_F_IND',
 'IF_ADD_L_IND',
 'IF_ADD_Q_IND',
 'IF_ADD_G_IND',
 'IF_ADD_R_IND',
 'IF_ADD_IND',
 'L1YR_PAYMENT_REMINDER_IND',
 'L1YR_LAPSE_IND',
 'LAST_B_CONTACT_DT',
 'A_IND',
 'B_IND',
 'C_IND',
 'LAST_C_DT',
 'IF_S_REAL_IND',
 'IF_Y_REAL_IND',
 'IM_IS_A_IND',
 'IM_IS_B_IND',
 'IM_IS_C_IND',
 'IM_IS_D_IND',
 'X_A_IND',
 'X_B_IND',
 'X_C_IND',
 'X_D_IND',
 'X_E_IND',
 'X_F_IND',
 'X_G_IND',
 'X_H_IND',
 'IF_HOUSEHOLD_CLAIM_IND',
 'IF_ISSUE_INSD_A_IND',
 'IF_ISSUE_INSD_B_IND',
 'IF_ISSUE_INSD_C_IND',
 'IF_ISSUE_INSD_D_IND',
 'IF_ISSUE_INSD_E_IND',
 'IF_ISSUE_INSD_F_IND',
 'IF_ISSUE_IN

In [30]:
#做label轉換
from sklearn.preprocessing import LabelEncoder

encoder1 = LabelEncoder() 

for i in range(0, len(label_encoder_feature)):
    feature = label_encoder_feature[i]
    data[feature] = encoder1.fit_transform(data[feature])
    
data.head()

,CUS_ID,GENDER,AGE,CHARGE_CITY_CD,CONTACT_CITY_CD,EDUCATION_CD,MARRIAGE_CD,LAST_A_CCONTACT_DT,L1YR_A_ISSUE_CNT,LAST_A_ISSUE_DT,...,IF_ADD_INSD_IND,L1YR_GROSS_PRE_AMT,CUST_9_SEGMENTS_CD,FINANCETOOLS_A,FINANCETOOLS_B,FINANCETOOLS_C,FINANCETOOLS_D,FINANCETOOLS_E,FINANCETOOLS_F,FINANCETOOLS_G
0,3418,M,低,A1,A1,3.0,0.0,1,0,0,...,0,0.000174,C,1,0,0,0,0,0,0
1,4302,M,低,A1,A1,3.0,0.0,1,0,0,...,0,0.008724,A,1,0,0,0,0,0,0
2,5545,M,低,A1,A1,1.0,0.0,1,1,1,...,0,0.005359,A,1,0,0,0,0,0,0
3,7207,M,低,A1,A1,3.0,0.0,1,0,0,...,0,0.000000,C,1,0,0,0,0,0,0
4,7213,M,低,A1,A1,1.0,0.0,0,0,0,...,0,0.000603,C,1,0,0,0,0,0,0


In [31]:
#做onehot轉換
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

encoder1 = LabelEncoder()
encoder2 = OneHotEncoder(sparse=False, categories='auto')

for i in range(0, len(onehot_encoder_feature)):
    feature = onehot_encoder_feature[i]
    data[feature] = encoder1.fit_transform(data[feature])
    b = encoder2.fit_transform(np.array(data[feature]).reshape(-1,1))
    data = pd.concat([data, pd.DataFrame(b).iloc[:, 1:]], axis=1)
data = data.drop(onehot_encoder_feature, axis = 1)
data.head()

,CUS_ID,LAST_A_CCONTACT_DT,L1YR_A_ISSUE_CNT,LAST_A_ISSUE_DT,L1YR_B_ISSUE_CNT,LAST_B_ISSUE_DT,CHANNEL_A_POL_CNT,CHANNEL_B_POL_CNT,APC_CNT,INSD_CNT,...,2,3,4,1,2,3,4,5,6,7
0,3418,1,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,4302,1,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,5545,1,1,1,0,0,1,0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,7207,1,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,7213,0,0,0,0,0,1,0,1,0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [32]:
#刪除ID
data.drop('CUS_ID', axis = 1, inplace = True)

# feature selection

### constant feature

In [33]:
#drop constant feature
from sklearn.feature_selection import VarianceThreshold

sel = VarianceThreshold(threshold=0)
sel.fit(data)

c:\Users\Julie\Documents\Python\.venv\lib\site-packages\sklearn\utils\validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(


VarianceThreshold(threshold=0)

In [34]:
print(
    len([
        x for x in data.columns
        if x not in data.columns[sel.get_support()]
    ]))
constant_feature = [x for x in data.columns if x not in data.columns[sel.get_support()]]
constant_feature

0


[]

In [35]:
data.drop(constant_feature, axis = 1, inplace = True)
data.shape

(250000, 165)

### quasi-constant feature

In [36]:
#drop quasi-constant feature
sel = VarianceThreshold(threshold=0.01)
sel.fit(data)

c:\Users\Julie\Documents\Python\.venv\lib\site-packages\sklearn\utils\validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(


VarianceThreshold(threshold=0.01)

In [37]:
print(
    len([
        x for x in data.columns
        if x not in data.columns[sel.get_support()]
    ]))

quasi_constant_feature = [x for x in data.columns if x not in data.columns[sel.get_support()]]
quasi_constant_feature

38


['L1YR_B_ISSUE_CNT',
 'LAST_B_ISSUE_DT',
 'IF_ISSUE_B_IND',
 'IF_ISSUE_E_IND',
 'IF_ISSUE_H_IND',
 'IF_ISSUE_K_IND',
 'IF_ISSUE_L_IND',
 'IF_ISSUE_M_IND',
 'IF_ADD_G_IND',
 'ANNUAL_PREMIUM_AMT',
 'ANNUAL_INCOME_AMT',
 'C_IND',
 'BANK_NUMBER_CNT',
 'BMI',
 'IM_IS_A_IND',
 'X_F_IND',
 'X_G_IND',
 'DIEBENEFIT_AMT',
 'DIEACCIDENT_AMT',
 'POLICY_VALUE_AMT',
 'ANNUITY_AMT',
 'EXPIRATION_AMT',
 'OUTPATIENT_SURGERY_AMT',
 'INPATIENT_SURGERY_AMT',
 'FIRST_CANCER_AMT',
 'ILL_ACCELERATION_AMT',
 'ILL_ADDITIONAL_AMT',
 'LONG_TERM_CARE_AMT',
 'MONTHLY_CARE_AMT',
 'LIFE_INSD_CNT',
 'IF_ISSUE_INSD_E_IND',
 'IF_ISSUE_INSD_H_IND',
 'IF_ISSUE_INSD_K_IND',
 'IF_ISSUE_INSD_L_IND',
 'IF_ISSUE_INSD_M_IND',
 'IF_ADD_INSD_G_IND',
 'L1YR_GROSS_PRE_AMT',
 'FINANCETOOLS_E']

In [38]:
data.drop(quasi_constant_feature, axis = 1, inplace = True)
#X_train = sel.transform(data)
data.shape

(250000, 127)

### duplicated feature

In [39]:
# check for duplicated features in the training set
duplicated_feat = []
for i in range(0, len(data.columns)):
    if i % 10 == 0:  # this helps me understand how the loop is going
        print(i)
    col_1 = data.columns[i]
    
    for col_2 in data.columns[i + 1:]:
        if data[col_1].equals(data[col_2]):
            duplicated_feat.append(col_2)
            
len(duplicated_feat)

0
10
20
30
40
50
60
70
80
90
100
110
120


161

In [40]:
data.drop(labels=duplicated_feat, axis=1, inplace=True)

### correlation feature

In [41]:
# find and remove correlated features
def correlation(dataset, threshold):
    col_corr = set()  # Set of all the names of correlated columns
    corr_matrix = dataset.corr()
    for i in range(len(corr_matrix.columns)):
        for j in range(i):
            if abs(corr_matrix.iloc[i, j]) > threshold: # we are interested in absolute coeff value
                colname = corr_matrix.columns[i]  # getting the name of column
                col_corr.add(colname)
    return col_corr
 
corr_features = correlation(data, 0.8)
print('correlated features: ', len(set(corr_features)) )

correlated features:  8


In [42]:
data.drop(labels=corr_features, axis=1, inplace=True)
data.shape

(250000, 73)

In [43]:
#onehot完會有column名稱重複，因此重新排序
data.columns = range(data.shape[1])

In [44]:
data[data.isnull().any(axis=1)].head()

,0,1,2,3,4,5,6,7,8,9,...,63,64,65,66,67,68,69,70,71,72


In [45]:
#切割資料
x_train=data.iloc[:100000,:]
x_test=data.iloc[100000:,:]
y_train=y.iloc[:100000]
y_test=y.iloc[100000:]

# 驗證集

In [46]:
##測roc用
from sklearn.model_selection import train_test_split

x_val_train, x_val_test, y_val_train, y_val_test = train_test_split(x_train, y_train, test_size=0.30,random_state=101)

# 模型

In [53]:
from xgboost import XGBClassifier

xgb = XGBClassifier()
xgb.fit(x_val_train,y_val_train)

c:\Users\Julie\Documents\Python\.venv\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[15:19:33] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
              gamma=0, gpu_id=-1, importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=12,
              num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [54]:
xgb_pred = xgb.predict(x_val_test)
xgb_pred

array([0., 0., 0., ..., 0., 0., 0.])

In [55]:
from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier(n_estimators=10)
rfc.fit(x_val_train,y_val_train)

RandomForestClassifier(n_estimators=10)

In [56]:
rfc_pred = rfc.predict(x_val_test)
rfc_pred

array([0., 0., 0., ..., 0., 0., 0.])

In [59]:
X = test.iloc[:,[7,9]]
X

,LAST_A_CCONTACT_DT,LAST_A_ISSUE_DT
0,Y,Y
1,N,N
2,N,N
3,N,N
4,Y,Y
...,...,...
149995,Y,N
149996,Y,Y
149997,Y,N
149998,Y,N


# 準確度測試

In [61]:
from sklearn.metrics import roc_auc_score

roc_auc_score(y_val_test, xgb_pred)

0.503162963375165

In [62]:
from sklearn.metrics import roc_auc_score

roc_auc_score(y_val_test, rfc_pred)

0.5074070982559047

# 匯出檔案

In [ ]:
a = test['CUS_ID']
b = pd.concat([a, pd.DataFrame(xgb_pred)], axis=1)
b.columns = ['CUS_ID', 'Ypred']
b.head()

In [ ]:
from pandas import DataFrame
df=DataFrame(b)
df.to_csv('submit_test.csv')